# 1-scraping

In [58]:
# Manejo de datos 
import pandas as pd # manejo de datos y dataframes
import numpy as np # manejo de arrays y operaciones matematicas 


# Librerias para realizar web scraping con selenium
from selenium import webdriver # webdriver permite manejar un navegador 
from webdriver_manager.chrome import ChromeDriverManager # permite instalar y mantener actualizado el driver de chrome
from selenium.webdriver.common.keys import Keys # permite simular teclas del teclado 
from selenium.webdriver.chrome.options import Options # permite configurar el driver de chrome como modo incognito o maximizar la ventana
from time import sleep # esperas entre ejecuciones de codigo


import warnings # permite ignorar los warnings de python 
warnings.filterwarnings('ignore')

In [59]:
import requests
import re
import pickle

from bs4 import BeautifulSoup

In [60]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])

# para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') # empezar maximizado
opciones.add_argument('user.data-dir=selenium') # guarda las cookies
opciones.add_argument('--incognito') # incognito window

# selenium

In [ ]:
urls = []
flag = 0
flag2 = 0

driver = webdriver.Chrome()

driver.get("https://www.dia.es/") 
sleep(3)

# aceptamos las cookies 
driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
sleep(2)

# clicka productos
driver.find_element("css selector", '#app > div > div > div > div.home-view__header > div.dia-header > div.dia-header__section.dia-header__section--start > div > div > button').click()
sleep(2)

# para cada una de las categorías principales
# poner un max excesivo
for category in range(1,31):

    while 1:

        # intenta clickar
        try:
            
            driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[{category}]/a').click()
            flag2 = 0

            for subcategory in range (1,31):

                while 1:

                    try:
                        
                        urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[{category}]/ul/div[{subcategory}]/a').get_attribute("href"))
                        sleep(0.1)
                        break

                    except:

                        flag2 = 1
                        break
                    
                if flag2:
                    break
            
            sleep(1)
            break
        
        # break si se acaban
        except:

            flag = 1
            break

    if flag:

        break

driver.quit()

In [62]:
df = pd.DataFrame(urls, columns=['url'])

df.to_csv("../data/urls-dia.csv", index=False, sep= ",")

# sopa bonita

In [63]:
# ['supermarket', 'category', 'name', 'description', 'price', 'reference_price', 'reference_unit', 'insert_date', 'product_id']

resultados_dia = {"supermarket": 'dia-es',
                "category":[],
                "name":[], 
                "description": [], 
                "price": [], 
                "reference_price": [],
                "reference_unit": [],
                "insert_date": [],
                "product_id": []}

In [64]:
url = 'https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001'
res = requests.get(url)
sleep(2)
#driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

print(res)

sopa = BeautifulSoup(res.content, 'html.parser')

productos = sopa.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
#productos = sopa.find_all("div", {"data-test-id": "product-card"})
productos = sopa.find_all("p", {"class": "search-product-card__product-name"})

print(f'Cantidad de productos extraídos: {len(productos)}')

print(productos[0])

print(productos[0].text)


<Response [200]>
Cantidad de productos extraídos: 10
<p class="search-product-card__product-name">Pechuga de pavo NUESTRA ALACENA pack 2 unidades SOBRE 400 GR</p>
Pechuga de pavo NUESTRA ALACENA pack 2 unidades SOBRE 400 GR
